In [1]:
import numpy as np
import cv2 as cv
import pygame
from ParticleFilter import ParticleFilter as pf

from filterpy.monte_carlo import systematic_resample
from numpy.random import randn, uniform
from math import dist
from fov_utils import fov_utils as fov
from main import simulation
from Line import Line

pygame 2.6.1 (SDL 2.28.4, Python 3.10.12)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [7]:
np.rad2deg(-np.pi/2)

np.float64(-90.0)

In [5]:
line = Line([0,2,2,2])

print(line.length)
print(line.slope)

2.0
0.0


In [ ]:
p1 = np.asarray([0,4])
p2 = np.asarray([3,0])
p3 = np.asarray([-5,5])

d = np.abs(np.cross(p2-p1, p1-p3))/np.linalg.norm(p2-p1)
print(d)
dist = np.linalg.norm(p1-p2)
print(dist)
print(p3[0])

0.25


In [ ]:
line  = [430, 394, 675, 394]
print(cv.norm((line[0],line[1]), (line[2],line[3])))

dist = dist([line[0],line[1]], [line[2],line[3]])
print(dist)
# Rho = (x1 * y2 - x2 * y1) / Len

245.0
245
245.0


In [2]:
N = 100
particles = np.empty((N, 3))
particles[:, 0] = uniform(0, 1300, size=N)
particles[:, 1] = uniform(0, 800, size=N)
particles[:, 2] = uniform(0, 360, size=N)
particles

array([[1292.86077056,  367.98774455,  102.54609579],
       [ 361.4966039 ,  571.5052983 ,   35.06821724],
       [ 115.6133719 ,  798.36394839,  267.5351174 ],
       [ 729.96730196,   61.55245696,   34.21392328],
       [ 355.13092373,  631.91239518,   35.61506288],
       [  28.78309418,  320.28478666,   74.69326162],
       [ 407.99667444,  702.64450544,  224.17466913],
       [1178.70025755,  595.88176208,  179.74713829],
       [ 678.34996364,  590.53872449,  311.2610805 ],
       [1053.62099069,  426.62928526,   76.5577612 ],
       [1137.03275515,  256.6112312 ,  352.55408789],
       [ 422.74832921,  701.34101477,  187.07777513],
       [  93.75144784,  202.86163971,  325.41914605],
       [ 207.09810121,  529.87034321,  222.53518308],
       [ 181.82379529,   32.28557323,  103.89593143],
       [  69.85494001,  326.29845251,  258.00349951],
       [ 459.34031215,  390.51752711,  173.06051895],
       [ 351.66030149,  256.97147905,  135.92061976],
       [1258.17391395,  575.

In [24]:

player_initial_x_deviation = 100 #Centimeters
player_initial_y_deviation = 100 #Centimeters
player_initial_angle_deviation = 10 #Degrees
standardDeviation = [player_initial_x_deviation,player_initial_y_deviation,player_initial_angle_deviation]

field_cv = cv.imread("images/soccer_field.jpg")
fov_max_range = 500 #centimeters
fov_min_range = 50 #centimeters
fov_angle = 87 #degrees

def create_gaussian_particles(N,initial_position = [0,0,0]):
    particles = np.empty((N, 3))
    particles[:, 0] = initial_position[0] + (randn(N) * standardDeviation[0])
    particles[:, 1] = initial_position[1] + (randn(N) * standardDeviation[1])
    particles[:, 2] = initial_position[2] + (randn(N) * standardDeviation[2])
    particles[:, 2] += 360
    particles[:, 2] %= 360
    return particles.astype(int)

particles = create_gaussian_particles(N = 10)

In [25]:
particles

array([[  -9,   41,  345],
       [ -87,  -74,  353],
       [-206,  173,    1],
       [  96,   66,  342],
       [ -49,   34,    7],
       [  79, -203,  344],
       [ -28,  -86,    3],
       [  59,  -97,    3],
       [  28,   94,   11],
       [  30, -125,    9]])

In [32]:
(np.sin(np.radians(particles[:,2])))*10

array([-2.58819045, -1.21869343,  0.17452406, -3.09016994,  1.21869343,
       -2.75637356,  0.52335956,  0.52335956,  1.90808995,  1.56434465])

In [33]:
(1 + randn(10) * 0.1)

array([0.9292214 , 1.03803735, 0.87310792, 0.9581732 , 1.20683914,
       1.13019259, 0.92225491, 0.88785863, 1.00939739, 1.02647999])

In [65]:

particles[:,1] -= np.multiply((np.sin(np.radians(particles[:,2])) * 10 * 1),(1 + randn(10) * 0.1)).astype(int)
particles

array([[  -9,  110,  345],
       [ -87,  -44,  353],
       [-206,  173,    1],
       [  96,  149,  342],
       [ -49,    3,    7],
       [  79, -134,  344],
       [ -28,  -86,    3],
       [  59,  -97,    3],
       [  28,   50,   11],
       [  30, -157,    9]])

In [15]:
player_initial_x = 200
player_initial_y = 300
player_initial_angle = 0
player_initial_x = 200
player_initial_y = 300
player_initial_angle = 0
player_pos = pygame.Vector2(player_initial_x, player_initial_y)
robot_fov = fov.get_particle_pov(field_cv, player_pos[0], player_pos[1], player_initial_angle, fov_angle, fov_min_range, fov_max_range, low_res = True)


In [16]:
def test_func(particle):
    particle_pov = fov.get_particle_pov(field_cv, particle[0], particle[1], particle[2], fov_angle, fov_min_range, fov_max_range, low_res = True)
    diff = 1/(np.sum(np.abs(np.subtract(robot_fov,particle_pov))))
    return diff

In [17]:
result1 = list(np.apply_along_axis(test_func,1,particles))
print(type(result1))
print(type(result1[0]))

<class 'list'>
<class 'numpy.float64'>


In [18]:
result2 = []
for particle in particles:
    diff = test_func(particle)
    result2.append(diff)

print(type(result2))
print(type(result2[0]))

<class 'list'>
<class 'numpy.float64'>


In [19]:
result3 = list(map(test_func,particles))
print(type(result3))
print(type(result3[0]))

<class 'list'>
<class 'numpy.float64'>


In [20]:
len(np.multiply(result1,result2))

100